In [7]:
# 📌 Upload your Kaggle API key first
from google.colab import files
files.upload()  # Upload kaggle.json from your Kaggle account

# 🔧 Set up Kaggle CLI
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 📥 Download the dataset
!kaggle datasets download -d snap/amazon-fine-food-reviews
!unzip -q amazon-fine-food-reviews.zip

# ✅ Install required packages
!pip install nltk scikit-learn pandas matplotlib seaborn wordcloud


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
License(s): CC0-1.0
 90% 217M/242M [00:00<00:00, 564MB/s] 
100% 242M/242M [00:00<00:00, 596MB/s]


In [8]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
nltk.download('punkt', force=True)
nltk.download('stopwords', force=True)
nltk.download('wordnet', force=True)
nltk.download('omw-1.4', force=True)
nltk.download('punkt_tab', force=True)
# 📥 Load dataset
df = pd.read_csv("Reviews.csv")[["Score", "Text"]]
df.dropna(inplace=True)

# 🎯 Sentiment labels
def label_sentiment(score):
    if score >= 4:
        return 'positive'
    elif score == 3:
        return 'neutral'
    else:
        return 'negative'

df["Sentiment"] = df["Score"].apply(label_sentiment)

# 🧹 Preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = re.sub(r"[^a-zA-Z]", " ", text).lower()
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

df["Cleaned_Text"] = df["Text"].apply(preprocess)

# ✂️ Split & Vectorize
X = df["Cleaned_Text"]
y = df["Sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer(max_features=20000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 🧠 Train model
model = LogisticRegression(max_iter=300)
model.fit(X_train_tfidf, y_train)

# 📈 Predict and Save
y_pred = model.predict(X_test_tfidf)

# Save models for later
joblib.dump(model, "sentiment_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump((X_test, y_test, y_pred), "test_data_and_predictions.pkl")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['test_data_and_predictions.pkl']

In [9]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pandas as pd
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

# 📁 Create folders
os.makedirs("Parameters", exist_ok=True)
os.makedirs("WeightsMatrix", exist_ok=True)

# 🔄 Load previous results
model = joblib.load("sentiment_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")
X_test, y_test, y_pred = joblib.load("test_data_and_predictions.pkl")

# 📊 Metrics Calculation
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# 💾 Save metrics to Parameters/
with open("Parameters/full_metrics_report.txt", "w") as f:
    f.write(f"Accuracy: {accuracy:.4f}\n")
    f.write(f"F1 Score: {f1:.4f}\n")
    f.write(f"Precision: {precision:.4f}\n")
    f.write(f"Recall: {recall:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(classification_report(y_test, y_pred))

report_df.to_csv("Parameters/classification_report.csv")

# 📉 Confusion Matrix Plot
cm = confusion_matrix(y_test, y_pred, labels=["positive", "neutral", "negative"])
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["positive", "neutral", "negative"],
            yticklabels=["positive", "neutral", "negative"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.savefig("Parameters/confusion_matrix.png")
plt.close()

# 📊 Precision, Recall, F1 Score per class
metrics = report_df.loc[["positive", "neutral", "negative"], ["precision", "recall", "f1-score"]]
metrics.plot(kind='bar', figsize=(8, 6), colormap='Set2')
plt.title("Precision, Recall & F1-score per Class")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig("Parameters/class_scores_barplot.png")
plt.close()

# 🧮 Support (sample count) per class
support = report_df.loc[["positive", "neutral", "negative"], "support"]
support.plot(kind='bar', figsize=(6, 4), color='teal')
plt.title("Support (Number of Samples) per Class")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig("Parameters/class_support_barplot.png")
plt.close()

# 📈 Sentiment Distribution: Actual vs Predicted
plt.figure(figsize=(6,4))
sns.countplot(x=y_test, label='Actual', alpha=0.6)
sns.countplot(x=y_pred, label='Predicted', alpha=0.6)
plt.title("Actual vs Predicted Sentiment Distribution")
plt.xlabel("Sentiment Class")
plt.legend()
plt.tight_layout()
plt.savefig("Parameters/sentiment_distribution.png")
plt.close()

# 💾 Save model & vectorizer
joblib.dump(model, "WeightsMatrix/sentiment_model.pkl")
joblib.dump(vectorizer, "WeightsMatrix/tfidf_vectorizer.pkl")

print("✅ All metrics and graphs saved in 'Parameters/', weights in 'WeightsMatrix/'")


✅ All metrics and graphs saved in 'Parameters/', weights in 'WeightsMatrix/'


In [10]:
import joblib
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# 📁 Replace these paths with your saved model/vectorizer paths
MODEL_PATH = "sentiment_model.pkl"
VECTORIZER_PATH = "tfidf_vectorizer.pkl"

# ✅ Load model and vectorizer
model = joblib.load(MODEL_PATH)
vectorizer = joblib.load(VECTORIZER_PATH)

# 🧹 Text Preprocessing Function
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt_tab', quiet=True)

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = re.sub(r"[^a-zA-Z]", " ", text).lower()
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

# 🔎 Test custom input
def test_review(review_text):
    cleaned = preprocess(review_text)
    vectorized = vectorizer.transform([cleaned])
    prediction = model.predict(vectorized)[0]
    print(f"\n📝 Original: {review_text}")
    print(f"🔮 Predicted Sentiment: **{prediction.upper()}**")

# 🧪 Example use:
test_review("I really love this product! It’s affordable and works perfectly.")
test_review("The food was cold and the service was terrible.")



📝 Original: I really love this product! It’s affordable and works perfectly.
🔮 Predicted Sentiment: **POSITIVE**

📝 Original: The food was cold and the service was terrible.
🔮 Predicted Sentiment: **NEGATIVE**
